In [1]:
import pandas as pd
import json
pd.set_option('display.max_columns', None) #Quitamos limitación de columnas máximas a mostrar
!cd

C:\Users\User\Documents\Master Data Science\SocialBigData\Twitter


### Importamos los conjuntos de entrenamiento y test: 
##### Hay cuatro categorías:
    Tráfico = T
    Contaminación = C
    Ambos = A
    Ninguno = N

In [2]:
train_tweets = pd.read_excel("./Data/train_tweets.xlsx", header=0, index_col='_id')
train_tweets.head(2)

,created_at,text,Category,Unnamed: 4
_id,,,,
791546620915486720,2016-10-27 07:46:35,#madrid alcanzado el escenario 1 del protocolo...,A,NaN
793885278553317376,2016-11-02 18:39:34,#madrid carmena propone restricciones al trafi...,A,NaN


In [3]:
test_tweets = pd.read_excel("./Data/test_tweets.xlsx", header=0, index_col='_id')
test_tweets.head(2)

,created_at,text,Category
_id,,,
801338536914022401,2016-11-23 08:16:10,#madrid marcha y concentracion ciclista por el...,A
842058467443630080,2017-03-15 17:02:37,nio eve: el coche autonomo (nivel 5) que se ca...,A


### Operaciones de tokenización, stemming y limpieza de texto:

In [4]:
%%time
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

cachedStopWords = stopwords.words("spanish")
stemmer = SnowballStemmer('spanish')

# Eliminamos carácteres no alfanuméricos:
# Eliminamos StopWords:
# Lematizamos:
train_tweets["SUMs"]=train_tweets["text"].apply(lambda x: re.sub(r'[^\w\s]','',str(x))).str.split()\
.apply(lambda x: list(set(filter(lambda y: y not in cachedStopWords,x))))\
.apply(lambda x: [stemmer.stem(token) for token in x])

test_tweets["SUMs"]=test_tweets["text"].apply(lambda x: re.sub(r'[^\w\s]','',str(x))).str.split()\
.apply(lambda x: list(set(filter(lambda y: y not in cachedStopWords,x))))\
.apply(lambda x: [stemmer.stem(token) for token in x])

Wall time: 1.86 s


In [5]:
train_tweets.sample(2)

,created_at,text,Category,Unnamed: 4,SUMs
_id,,,,,
751373952392593408,2016-07-08 11:14:44,#incdgt el 08-07-2016 09:00:00 tipo #obra (obr...,T,136.0,"[obras, 090000, m50, 08072016, pk33_6, kilomet..."
770233905106608132,2016-08-29 12:17:27,#incdgt el 29-08-2016 09:30:00 tipo #obra (man...,T,NaN,"[crecient, coll, puent, d, manten, 29082016, i..."


### Creamos diccionario de palabras únicas y número de tweets en los que aparecen:

In [6]:
%%time
diccio = {}
for SUM in train_tweets.append(test_tweets)["SUMs"]:
    for word in SUM:
        if word in diccio.keys():
            diccio[word]+=1
        else: diccio[word]=1
print(len(diccio))

5460
Wall time: 29.1 ms


### Creamos diccionario de palabras con IDFs:

In [7]:
import numpy as np

idfs={}
n_docs = len(train_tweets.append(test_tweets))

for key in diccio.keys():
    idfs[key]= np.log10(n_docs/diccio[key])
print(len(idfs))

5460


In [8]:
idfs["trafic"]

2.2894774663446023

### Definimos función para crear vector de características para cada tweet:

In [9]:
def feat_vect_creat(vect, diccionario):
    """Creación del vector de características"""
    feat_vect = []
    for feat in diccionario.keys():
        if feat in vect:
            feat_vect.append(diccionario[feat])
        else:
            feat_vect.append(0)
    return feat_vect

### Creamos los vectores de características de cada tweet para los sets de entrenamiento y test:

In [10]:
%%time

train_tweets["feature_vector"] = train_tweets["SUMs"].map(lambda x: feat_vect_creat(x, idfs))
test_tweets["feature_vector"] = test_tweets["SUMs"].map(lambda x: feat_vect_creat(x, idfs))

Wall time: 2.35 s


In [11]:
train_tweets.head(2)

,created_at,text,Category,Unnamed: 4,SUMs,feature_vector
_id,,,,,,
791546620915486720,2016-10-27 07:46:35,#madrid alcanzado el escenario 1 del protocolo...,A,NaN,"[contaminacion, alcanz, protocol, restriccion,...","[1.52980962165, 3.19256745334, 2.11338620729, ..."
793885278553317376,2016-11-02 18:39:34,#madrid carmena propone restricciones al trafi...,A,NaN,"[madridâ, httpstcotqybs8mv8c, restriccion, cen...","[0, 0, 0, 2.41441620295, 0, 0, 0, 0, 0, 2.2894..."


### Extraemos los vectores de características en listas para a continuación crear DataFames de características:

In [12]:
%%time

train_vectors = [vector for vector in train_tweets["feature_vector"].values]
test_vectors = [vector for vector in test_tweets["feature_vector"].values]

train_df = pd.DataFrame(train_vectors, columns=idfs.keys(), index=train_tweets.index)
test_df = pd.DataFrame(test_vectors, columns=idfs.keys(), index=test_tweets.index)

print(test_df.shape)

(222, 5460)
Wall time: 1.52 s


### Definimos funciones para obtener la ganancia de cada raíz

In [13]:
def entropy(attribute_data):
    """
    Calcula la entropía de cada palabra
    Recibe un vector y devuelve un flotante
    """
    _, val_freqs = np.unique(attribute_data, return_counts=True)
    # probabilities for each unique attribute value
    val_probs = val_freqs / len(attribute_data)
    return -val_probs.dot(np.log(val_probs))
print(entropy(train_df["posibl"]))
print(entropy(train_df["trafic"]))

0.00613552470748
0.0327412470058


In [14]:
def get_count_dict(data):
    """
    Devuelve valores únicos y su frecuencia en un diccionario, para cada vector (de raiz) proporcionado
    """
    data_values, data_freqs = np.unique(data, return_counts=True)
    return dict(zip(data_values, data_freqs))

In [15]:
def info_gain(attribute_data, labels):
    """
    Calcula la ganancia de información
    Recibe un vector de caracteristicas para una raíz, y un vector de etiquetas
    Devuelve la ganancia como valor flotante
    """
    attr_val_counts = get_count_dict(attribute_data)
    total_count = len(labels)
    EA = 0.0
    for attr_val, attr_val_count in attr_val_counts.items():
        EA += attr_val_count * entropy(labels[attribute_data == attr_val])


    return entropy(labels) - EA / total_count

print(info_gain(train_df["posibl"],train_tweets["Category"]))
print(info_gain(train_df["trafic"],train_tweets["Category"]))

0.00355915360218
0.0105690883726


### Creamos diccionario de ganancia de las raíces 

In [16]:
%%time
ganancias={}
for key in idfs.keys():
    ganancias[key]= info_gain(train_df[key],train_tweets["Category"])

Wall time: 10 s


### Creamos diccionario de las raíces con mayor ganancia
#### Utilizamos como criterio las que tienen ganancias por encima de la media

In [17]:
%%time
media = np.mean(list(ganancias.values()))
relevant_stems = {}
for key in ganancias.keys():
    if ganancias[key]>media:
        relevant_stems[key] = ganancias[key]
print(len(ganancias.keys()))
print(len(relevant_stems.keys()))

5460
595
Wall time: 1 ms


### Reconstruimos los vectores de características para sólo las palabras relevantes

In [18]:
%%time

#Creación de los vectores en los DataFrames
train_tweets["feature_vector"] = train_tweets["SUMs"].map(lambda x: feat_vect_creat(x, relevant_stems))
test_tweets["feature_vector"] = test_tweets["SUMs"].map(lambda x: feat_vect_creat(x, relevant_stems))

#Extracción de los vectores
train_vectors = [vector for vector in train_tweets["feature_vector"].values]
test_vectors = [vector for vector in test_tweets["feature_vector"].values]

print(len(train_vectors[0]))
print(len(test_vectors[0]))

#Creación de DataFrames de características
train_df = pd.DataFrame(train_vectors, columns=relevant_stems.keys(), index=train_tweets.index)
test_df = pd.DataFrame(test_vectors, columns=relevant_stems.keys(), index=test_tweets.index)

595
595
Wall time: 469 ms


In [19]:
print(train_df.shape)
train_df.head(2)

(1336, 595)


,contaminacion,alcanz,protocol,restriccion,1,posibl,escenari,28,httpstco6dho7ojtpf,trafic,madr,viern,madridâ,httpstcotqybs8mv8c,centr,carmen,resident,propon,anunci,httpstcot40hzmua2z,aparc,circulacion,12,delââ,httpstcoetldkovpdc,concieloazul,estoesmadridâ,conestaluz,sintraf,sinmaloshum,httpstcob8emmrht2i,distintivoambiental,mostol,coch,ecofriendly,dgt,fog,spain,carabany,httpstcody37a80vrn,niebl,mas,50,httpstcognoqnaw2jx,httpstcopem4r4veoj,contamin,pag,gratis,experient,aparcadondequ,primer,â,emov,httpstcoijuwqoabmt,nocontamin,feliz,httpstcozlrwhaupo,gran,diciembr,air,mejor,cierr,calid,32,parcial,viaâ,levels,ban,httpstcozvbyoz4jfi,from,half,to,pollution,due,the,city,be,will,tim,cars,cent,this,thatâ,first,of,high,ecologicosâ,parquimetr,subir,except,preci,httpstcoaoqmwdmx3w,buen,afin,part,sol,ahoramadr,regulacion,via,ide,qued,manuelacarmen,tragic,peatonaliz,gan,call,sky,demadridalciel,public,dias,ayunt,sig,llov,siempr,palac,gris,posicion,tendenci,trndnl,ser,httpstcok5dfqqcsew,ocup,espany,ahor,parqu,delâ,comun,âqu,anticontaminacion,veloc,m30,restring,activ,no2,zon,cas,sab,contaminacionâ,nivel,nuev,fri,pobrez,crisis,chin,blackandwhit,blancoynegr,in,alertacontaminacionep,for,ciel,sierr,vist,convert,asi,nub,boin,torrelodon,anochec,madrid,metr,precios,ciud,polucion,verd,vam,pur,si,puert,salud,hac,diput,rio,sinfiltr,san,dia,cos,aqui,nightâ,noch,nofilt,matader,nofilters,realmadr,madridmemol,sinfiltrosâ,nsinfiltr,hotel,alnatural,nofilterâ,sunset,atardec,curr,graci,calle30,megust,amanecer,mund,templ,debod,tard,byn,lluvi,i,lleg,sinfiltroâ,espectacul,com,arquitectur,monclo,chamberi,madridmol,arroyofresn,elcielodemadr,mirasierr,bychinygb,photobychinygb,real,palaci,jardin,pase,eurotravelâ,madridsinfiltr,luz,puestadesol,otony,retir,doming,ðÿ,ðÿœ,camp,primaver,color,final,sun,bonit,amanec,sunris,retaz,atardecer,obras,deâ,buenosdi,mont,pas,navid,unviajesinequipajeâ,fern,disfrut,ven,im,lov,natural,roz,henar,alcal,you,aeropuert,mad,estadi,bernabeu,santiag,hoy,segu,trabaj,instagram,filtr,travel,beautiful,dur,pint,roj,alcorcon,despu,boadill,precaucion,m40,23052016,incdgt,blanc,tip,decrecient,kilometr,crecient,alcobend,m100,pk25_0,27042016,kilom,accident,m45,pk0_0,17062016,25052016,pk9_0,a1,07072016,pk5_0,a2,100000,carril,a6,pk24_0,pk34_0,16062016,rey,amarill,071700,sebasti,pk17_0,decreci,090000,22112016,m505,pk20_0,general,obra,084200,pk10_0,pk43_0,23112016,080000,25042016,pk12_0,pk6_0,080200,13062016,20062016,18052016,26082016,d,manten,bici,m601,reasfalt,pk20_2,navacerr,093000,pk,a42,parl,m607,kil,tres,30112016,pk16_0,cant,ki,230000,m50,29042016,kilomet,14032016,pk18_0,pk8_3,pk32_0,20042016,a4,29062016,a3,080300,06052016,pk4_5,a5,pk29_0,21112016,25112016,pk7_0,03102016,29082016,223000,16032016,183300,pozuel,l,pk12_5,alarcon,pk28_0,21062016,17112016,pk33_0,14072016,pk14_0,17052016,torrejon,pk23_0,ardoz,31032016,pk13_2,27062016,23062016,30092016,contrari,01122016,pk11_0,02122016,k,18112016,09052016,madbikestatus,bicim,dispon,rivasvaciamadr,10052016,pk15_0,pk51_0,26072016,m11,29112016,averi,pk15_5,pk27_0,pk13_0,m423,valdemor,pk8_0,m503,071300,15072016,pk22_0,ap6,24062016,28112016,000000,pk21_0,pk4_0,29092016,12052016,01072016,guadalix,08072016,04102016,073200,m512,27052016,24052016,05102016,agustin,pk30_0,calz,senyalizacion,m222,valdaracet,20052016,181900,13072016,decr,26052016,28062016,pk49_5,13052016,13122016,20072016,pk19_0,05072016,creci,pk48_0,dgtmadr,12042016,pk9_5,11052016,m513,10112016,22062016,15032016,10082016,estadioâ,halamadr,instal,cest,vem,menu,fot,hech,cam,and,was,iberi,travelgram,cambi,turiste,that,all,httpstcolvso6nplxn,corr,fiest,tourismâ,business,egaylity,igersspain,world,at,calderon,barc,vicent,almuerz,pon,mil,luch,champions,verano2016,quer,padr,foll,parej,amor,hal,architectur,prid,gay,my,vip,phot,tom,travelblogg,chiringuitomadridn4,traveling,atlet,atleti,2017,listasdepuert,discotecasmadr,albertodelacru,restaur,mayor,santiagobernabeu,executiv,nsx,bas,httpstco0j4bplq2ln,job,account,vacation,homosexual,turism,famili,ðÿªðÿ,gust,plat,fa

### Implementación de SVM con kernel predefinido:

In [20]:
%%time

from sklearn.svm import SVC
from sklearn import metrics
svc=SVC() #Default hyperparameters
svc.fit(train_df,train_tweets["Category"])
y_pred=svc.predict(test_df)
print('Accuracy Score: %f' %metrics.accuracy_score(test_tweets["Category"],y_pred))

Accuracy Score: 0.761261
Wall time: 2.01 s


### Implementación de SVM con kernel lineal:

In [21]:
svc=SVC(kernel='linear')
svc.fit(train_df,train_tweets["Category"])
y_pred=svc.predict(test_df)
print('Accuracy Score: %f' %metrics.accuracy_score(test_tweets["Category"],y_pred))

Accuracy Score: 0.765766


In [22]:
print(y_pred)
print(test_tweets["Category"].values)

['N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'C' 'N' 'N' 'N' 'N' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'N'
 'T' 'T' 'T' 'N' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T'
 'T' 'T' 'T' 'T' 'N' 'T' 'T' 'T' 'N' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T'
 'T' 'T' 'T' 'T' 'T' 'T' 'N' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T'
 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'N' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'N'
 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'T' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N']
['A' 'A' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C' 'C'
 'C' 'C' 'C' 'C' 'C' 'C' 

### Cargamos el archivo completo de tweets

In [23]:
%%time

"""Cargo los tweets en una lista, algunas líneas no son tweets completos, 
sino sólo trozos y no es posible cargarlos e incorporarlos al DataFrame, por eso try-except.
Al final con esto lastimosamente dejan de cargarse muchos tweets"""

with open("./Data/CAM20170501-002.json") as data:
    n=0
    m=0
    jr=0
    js=0
    l1=[]
    l2=[]
    l3=[]
    for line in range(1587572):
        try:
            r1=(data.readline())
        except:
            n+=1
            try:
                l1.append(r1)
            except:
                pass
        else: 
            try:
                r2=json.loads(r1)
            except:
                jr+=1
                try:
                    r2=json.loads(json.dumps(r1))
                except:
                    js+=1
                else:
                    m+=1
                    l3.append(r2)
            else:
                m+=1
                l2.append(r2)
    print("Cargados=%d\nPerdidos al leer linea= %d\nPerdidos al cargar json raw = %d\nPerdidos al cargar json string = %d"\
          %(m,n,jr,js))
    print("L1 = %d\nL2 = %d\nL3 = %d" %(len(l1),len(l2), len(l3)))

Cargados=1394502
Perdidos al leer linea= 193070
Perdidos al cargar json raw = 736206
Perdidos al cargar json string = 0
L1 = 193070
L2 = 658296
L3 = 736206
Wall time: 1min 45s


In [24]:
%%time
"""Aquí cargamos los tweets en un DataFrame"""

ll1 = []
for element in l1:
    try:
        ll1.append(json.loads(element))
    except:
        pass
tweets1 = pd.DataFrame(ll1)

Wall time: 42.6 s


In [25]:
print(tweets1.shape)
tweets1.head(2)

(192780, 54)


,Xcoord,Ycoord,_id,coordinates,created_at,created_at_localtime,display_text_range,entities,entities-hashtags,entities-media,entities-urls,entities-user_mentions,entitites,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,in_reply_to_screen_name,in_reply_to_screen_nameFix,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,place-country_code,place-name,possibly_sensitive,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,source,sourceFix,text,textFix,timestamp_ms,timezone_info,truncated,user,user-created_at,user-followers_count,user-friends_count,user-id_str,user-lang,user-listed_count,user-location,user-screen_name,user-statuses_count
0,-3.7058,40.4203,722257977667747843,"{'type': 'Point', 'coordinates': [-3.7058, 40....",{'$date': '2016-04-19T02:58:15.000Z'},{'$date': '2016-04-19T04:58:15.000Z'},NaN,"{'hashtags': [{'indices': [121, 128], 'text': ...","[{'indices': [121, 128], 'text': 'trndnl'}]",NaN,"[{'url': 'https://t.co/K5DFqqcseW', 'indices':...",NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,"{'name': 'Madrid', 'country_code': 'ES'}",ES,Madrid,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,"'Oklahoma' empieza a ser Tendencia, con 16652 ...",NaN,NaN,"{'timezone': 'Europe/Madrid', 'obtained_from':...",NaN,"{'lang': 'es', 'statuses_count': 432224.0, 'sc...",{'$date': '2013-02-14T18:39:28.000Z'},36306.0,44.0,1179981192,es,450.0,EspaÃ±a,trendinaliaES,432224.0
1,-3.7058,40.4203,722269265693310977,"{'type': 'Point', 'coordinates': [-3.7058, 40....",{'$date': '2016-04-19T03:43:06.000Z'},{'$date': '2016-04-19T05:43:06.000Z'},NaN,"{'hashtags': [{'indices': [0, 12], 'text': 'Lu...","[{'indices': [0, 12], 'text': 'LunesDeOjos'}, ...",NaN,"[{'url': 'https://t.co/M8vQMwWlAW', 'indices':...",NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,es,"{'name': 'Madrid', 'country_code': 'ES'}",ES,Madrid,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,#LunesDeOjos estuvo el lunes 18 como Tendencia...,NaN,NaN,"{'timezone': 'Europe/Madrid', 'obtained_from':...",NaN,"{'lang': 'es', 'statuses_count': 432234.0, 'sc...",{'$date': '2013-02-14T18:39:28.000Z'},36306.0,44.0,1179981192,es,450.0,EspaÃ±a,trendinaliaES,432234.0


In [26]:
tweets3 = pd.DataFrame(l3)
tweets2 = pd.DataFrame(l2)

print(tweets1.shape)
print(tweets2.shape)
print(tweets3.shape)

(192780, 54)
(658296, 54)
(736206, 1)


In [27]:
del tweets3, l1, l2, l3, ll1, data

In [28]:
tw = tweets2.append(tweets1) #formamos un df único
print(tw.shape)
tw = tw.set_index("_id")
print(tw.shape)

(851076, 54)
(851076, 53)


In [29]:
del tweets2, tweets1

In [30]:
tw["texto"]=tw["text"]

### Reemplazamos carácteres extraños por letras simples
(á = Ã¡ , á = Ã\xa0 , É = Ã‰ , é = Ã© , é = Ã¨ , í = Ã\xad , í = Ã¬ , Ó = Ã“ , Ó = Ã’ , ó = Ã³ , Ú = Ãš , ú = Ãº , ü = Ã¼, Ñ = Ã‘ , ñ = Ã± , Ç = Ã‡ , ç = Ã§) 

In [31]:
%%time
tw["text"] = tw["text"].str.replace(r"\Ã\¡","a")
tw["text"] = tw["text"].str.replace(r"\Ã\xa0","a")
tw["text"] = tw["text"].str.replace(r"\Ã\©","e")
tw["text"] = tw["text"].str.replace(r"\Ã\¨","e")
tw["text"] = tw["text"].str.replace(r"\Ã\xad","i")
tw["text"] = tw["text"].str.replace(r"\Ã\¬","i")
tw["text"] = tw["text"].str.replace(r"\Ã\²","o")
tw["text"] = tw["text"].str.replace(r"\Ã\º","u")
tw["text"] = tw["text"].str.replace(r"\Ã\¼","u")
tw["text"] = tw["text"].str.replace(r"\Ã\±","ny")
tw["text"] = tw["text"].str.replace(r"\Ã\§","c")

tw["text"] = tw["text"].str.replace(r"\Ã\‰","E")
tw["text"] = tw["text"].str.replace(r"\Ã\“","O")
tw["text"] = tw["text"].str.replace(r"\Ã\’","O")
tw["text"] = tw["text"].str.replace(r"\Ã\š","U")
tw["text"] = tw["text"].str.replace(r"\Ã\‘","NY")
tw["text"] = tw["text"].str.replace(r"\Ã\‡","C")

tw["text"] = tw["text"].str.lower() # Convertimos a minúsculas

Wall time: 15.5 s


### Tokenización, limpieza y Lematización

In [32]:
%%time
# Eliminamos carácteres no alfanuméricos:
# Eliminamos StopWords:
# Lematizamos:
tw["SUMs"]=tw["text"].apply(lambda x: re.sub(r'[^\w\s]','',str(x))).str.split()\
.apply(lambda x: list(set(filter(lambda y: y not in cachedStopWords,x))))\
.apply(lambda x: [stemmer.stem(token) for token in x])

Wall time: 9min 24s


### Creación de vectores y DataFrame de características:

In [ ]:
%%time

tw["feature_vector"] = tw["SUMs"].map(lambda x: feat_vect_creat(x, relevant_stems))
vectores = [vector for vector in tw["feature_vector"].values]
features_df = pd.DataFrame(vectores, columns=relevant_stems.keys(), index=tw.index)

### Etiquetado de tweets
#### Primero normalizamos los vectores

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(features_df)
features_df = scaler.transform(features_df)

In [ ]:
tw["category"] =pd.DataFrame(svc.predict(features_df))

### Exportamos los tweets etiquetados a csv

In [ ]:
tw.to_csv('./Data/labeled_tweets.csv', sep='¡')

In [ ]:
analytic_tweets = tw[["created_at","text", "lang", "Xcoord","Ycoord","SUMs","category"]]

In [ ]:
del tw

In [ ]:
analytic_tweets.to_csv('./Data/analytic_tweets.csv', sep='¡')